# Template Definition

In [1]:
ROOT_FOLDER = ""

In [2]:
prompt_template_class = """
Step 1:

You will be asked to extact, from text specification, the list of class to use in 
a uml conceptual model diagram. Include only classes relevant for conceptual modeling.

Choose the key words from text and output the results in a list form:

[ClassName1, ClassName2, ...]

i.e. for a Book text specification:

[Book, Page, Author, ...]

Output only the list and no other text.
Keep the list as short as possible and do not add irrelevant classes.

##############

The specification text is: 

{text}.

##############

The class list is:
"""

prompt_template_assoc =  """
Step 2:

You will be asked to extact, from text specification and a list of classes, the associations amongst
the classes to use in a uml conceptual model diagram. 

Include only associations relevant for conceptual modeling. Use only classes from the list.


Choose the key words from class list according to text specifications and output the results in a list form:

[(ClassName1, ClassName2), (ClassName1,Classname3), ...]

i.e. for a Book text specification:

[(Book, Page), (Book, Author), ...]

All associations are meant to be symmetrical. If (ClassName1, ClassName2) is in the association list,
there is no need to add (Classname2, ClassName1).

Output only the list and no other text.
Keep the list as short as possible and do not add irrelevant association.

##############

The class list is: 

{classes}

#############

The specification text is: 

{text}

##############

The association list is:
    
"""


prompt_template_card = """
Step 3:

You will be asked to extact, from text specification and a list of association, the cardinality for 
each association to use in a uml conceptual model diagram. 

Use only association from the list. The format of the cardinality is:

min_cardinality..max_cardinlaity

For example 1..100 has 1 as minimum cardinality and 100 as maximum cadinality.
Use the asterisk (*) for an undefined number of maximum cardinality,
and 0 for an undefined number of minumum cardinality.

If you are not sure assign default cardinality: 0..*

Decide the correct cardinality for each association from text and output the results in a list form:

[(ClassName1, Cadinality1 ,ClassName2, Cardinality2), ...]

i.e. for a Book text specification:

[(Book, 1..* ,Page, 1..1),  ...]

Output only the list and no other text.

##############

The association list is:

{association}

#############

The specification text is: 

{text}

##############

The cardinality list is:
"""
    

prompt_template_plant = """
Step 4:

You will be asked by the user to create a plant UMl model from a class list and a cardinality list.
Do so in the most clear way possible, avoid class properties and assign molteplicity. 

The format of class list is:

[ClassName1, ClassName2, ...]

The format of association list is:

[(ClassName1, Cadinality1 ,ClassName2, Cardinality2), ...]

for every class in class list write:

class ClassName1{{}}
class ClassName2{{}}

for every tuple in association list write:

ClassName1 "Cardinality1" -- "Cardinality2" Classname2.

Output plantuml without futher text or explaination. Use tags 
@startuml
at the beginning and 
@enduml 
at the end of the output.

##############

The class list is: 

{classes}

#############

##############

The cardinality list is:

{card}

#############

The plant uml is:
                                    
"""

In [3]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [4]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama

In [5]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [6]:
def build_chian_from_llms(llm):
    prompt_class = PromptTemplate(
        input_variables=["text"],
        template = prompt_template_class                     
    )
    
    prompt_assoc = PromptTemplate(
        input_variables=["text","classes"],
        template = prompt_template_assoc                    
    )
    
    prompt_card = PromptTemplate(
        input_variables=["text","association"],
        template = prompt_template_card                  
    )
    
    prompt_uml = PromptTemplate(
        input_variables=["classes","association"],
        template = prompt_template_plant                
    )
    
    class_chain = prompt_class | llm | StrOutputParser()
    assoc_chain = prompt_assoc | llm | StrOutputParser()
    card_chain  =  prompt_card | llm | StrOutputParser()
    uml_chain   =   prompt_uml | llm | StrOutputParser()
    
    full_chain = (
         RunnableLambda(lambda x: {"text": x["text"]})
        | {
            "text": lambda x: x["text"],
            "classes": class_chain
        }
        | {
            "text": lambda x: x["text"],
            "classes": lambda x: x["classes"],
            "association": assoc_chain
        }
        | {
            "text": lambda x: x["text"],
            "association": lambda x: x["association"],
            "classes": lambda x: x["classes"],
            "card": card_chain
        }
        | {
            "classes": lambda x: x["classes"],
            "card": lambda x: x["card"],
            "uml": uml_chain
        } 
        | RunnableLambda(lambda x: x["uml"])
        | StrOutputParser()
    )
    return full_chain


In [7]:
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function(fn_name):
    # print to stderr, unbuffered in Python 2.
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt
    
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [8]:
import os
from tqdm import tqdm
from docx import Document

from langchain_core.tracers.context import tracing_v2_enabled

#@exit_after(360)
def run_chain(chain, text):
    return chain.invoke({"text": text})
    
def process_subfolders_with_chain(root_folder_path, chain, type=''):
    """
    Explores subfolders of the root folder (depth 1), processes each subfolder's `text.txt`
    with the provided LangChain chain, and saves the result in a new file in the same folder.

    Args:
        root_folder_path (str): Path to the root folder.
        chain: A LangChain chain instance to process text inputs.
    """
    for subfolder_name in tqdm(os.listdir(root_folder_path)):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            text_file_path = os.path.join(subfolder_path, "text.txt")
            
            # Check if `text.txt` exists in the subfolder
            if not os.path.isfile(text_file_path):
                # If `text.txt` is missing, check for any .docx file in the subfolder
                docx_files = [f for f in os.listdir(subfolder_path) if f.endswith(".docx")]
                if docx_files:
                    docx_file_path = os.path.join(subfolder_path, docx_files[0])
                    # Extract content from the .docx file
                    doc = Document(docx_file_path)
                    text_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

                    # Save the extracted content to `text.txt`
                    with open(text_file_path, "w", encoding="utf-8") as text_file:
                        text_file.write(text_content)

            # Recheck if `text.txt` now exists
            if os.path.isfile(text_file_path):
                with open(text_file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                with tracing_v2_enabled():
                    # Call the LangChain chain with the input dictionary
                    try:
                        result = run_chain(chain, text)
                    except:
                        reuslt = ""

                # Save the result to a new file in the same subfolder
                result_file_path = os.path.join(subfolder_path, f"result_cot_{type}.txt")
                with open(result_file_path, "w", encoding="utf-8") as result_file:
                    result_file.write(result)

In [9]:
def delete_result_txt_files(root_folder_path):
    """
    Deletes every .txt file that starts with 'result_' in the subfolders of the root folder (depth 1).

    Args:
        root_folder_path (str): Path to the root folder.
    """
    for subfolder_name in os.listdir(root_folder_path):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            for file_name in os.listdir(subfolder_path):
                if file_name.startswith("result_") and file_name.endswith(".txt"):
                    file_path = os.path.join(subfolder_path, file_name)
                    os.remove(file_path)

In [10]:
#delete_result_txt_files(ROOT_FOLDER)

# COT Open-AI

In [10]:
from dotenv import load_dotenv
assert load_dotenv()

In [10]:
MODEL_OPEN_AI = ["o3-mini","gpt-4o-mini", "gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"]

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.chat.output_parser import ChatOutputParser
from IPython.display import display_markdown

In [12]:
def open_ai_zero(model):
    model_ = ChatOpenAI(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [13]:
def open_ai_make_example(model):
    model = ChatOpenAI(model=model)
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [14]:
display_markdown(open_ai_make_example(MODEL_OPEN_AI[0]), raw=True)

@startuml
class Customer {}
class Broker {}
class InsurancePolicy {}
class Offer {}
class Contract {}
class Invoice {}
class ClaimCase {}
class Estimator {}
class Report {}
class CompensationDecision {}
class Payment {}

Customer "1..1" -- "0..*" Broker
Customer "1..*" -- "0..*" InsurancePolicy
Customer "0..*" -- "1..1" Offer
Customer "0..*" -- "1..1" Contract
Customer "0..*" -- "1..1" Invoice
Customer "0..*" -- "1..1" ClaimCase
ClaimCase "1..*" -- "0..*" Estimator
Estimator "0..*" -- "1..1" Report
ClaimCase "1..*" -- "1..1" Report
ClaimCase "0..1" -- "1..1" CompensationDecision
Customer "0..*" -- "1..1" Payment
@enduml

In [15]:
for model in MODEL_OPEN_AI[0:1]:
    print(f"COT with {model}")
    open_ai_zero(model)

COT with o3-mini


100%|███████████████████████████████████████████| 48/48 [29:02<00:00, 36.30s/it]


# COT Open LLM

In [24]:
import torch, gc
gc.collect()
torch.mps.empty_cache()

## Deepseek

In [15]:
from langchain_deepseek import ChatDeepSeek

In [16]:
MODEL_DEEPSEEK = ["deepseek-chat"] #, "deepseek-reasoner"]

In [20]:
def deepseek_make_example(model):
    model = llm = ChatDeepSeek(
            model=model,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            # api_key="...",
            # other params...
        )
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [18]:
def deepseek_cot(model):
    model_ = ChatDeepSeek(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [21]:
%%time
display_markdown(deepseek_make_example(MODEL_DEEPSEEK[0]), raw=True)

```plantuml
@startuml
class InsuranceCompany{}
class Customer{}
class Broker{}
class InsurancePolicy{}
class Contract{}
class Claim{}
class ClaimCase{}
class Estimator{}
class CompensationDecision{}
class Invoice{}

InsuranceCompany "1..1" -- "0..*" Customer
Customer "1..1" -- "1..1" Broker
Customer "1..*" -- "0..*" InsurancePolicy
Broker "1..1" -- "0..*" InsurancePolicy
InsurancePolicy "1..1" -- "1..1" Contract
Customer "1..*" -- "0..*" Claim
Claim "1..1" -- "1..*" ClaimCase
ClaimCase "1..1" -- "1..*" Estimator
ClaimCase "1..1" -- "1..1" CompensationDecision
Contract "1..1" -- "1..*" Invoice
@enduml
```

CPU times: user 254 ms, sys: 48 ms, total: 302 ms
Wall time: 54.6 s


In [22]:
for model in MODEL_DEEPSEEK:
    print(f"COT with {model}")
    deepseek_cot(model)

CoT shot with deepseek-chat


100%|███████████████████████████████████████████| 48/48 [31:46<00:00, 39.71s/it]


## Anthropic

In [24]:
from langchain_anthropic import ChatAnthropic

In [25]:
MODEL_ANTHROPIC = ["claude-3-7-sonnet-20250219"]

In [26]:
def anthropic_make_example(model):
    model = ChatAnthropic(model=model,temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,)
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [27]:
def anthropic_cot(model):
    model_ = ChatAnthropic(model=model)
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [28]:
display_markdown(anthropic_make_example(MODEL_ANTHROPIC[0]), raw=True)

@startuml
class Customer{}
class Broker{}
class InsurancePolicy{}
class Contract{}
class Offer{}
class Invoice{}
class Claim{}
class ClaimCase{}
class Estimator{}
class Report{}
class CompensationDecision{}
class Payment{}

Customer "1..1" -- "1..*" Broker
Customer "1..*" -- "0..*" InsurancePolicy
Customer "1..*" -- "1..*" Contract
Customer "1..*" -- "0..*" Offer
Customer "1..*" -- "0..*" Invoice
Customer "1..*" -- "0..*" Claim
Customer "1..*" -- "0..*" Payment
Broker "1..*" -- "0..*" Contract
Broker "1..*" -- "0..*" Offer
InsurancePolicy "1..1" -- "1..1" Contract
InsurancePolicy "1..1" -- "0..*" Invoice
Contract "1..1" -- "0..1" Offer
Claim "1..1" -- "1..*" ClaimCase
ClaimCase "1..*" -- "1..*" Estimator
ClaimCase "1..1" -- "0..1" CompensationDecision
Estimator "1..*" -- "1..*" Report
CompensationDecision "1..1" -- "0..1" Payment
@enduml

In [30]:
for model in MODEL_ANTHROPIC:
    print(f"COT with {model}")
    anthropic_cot(model)

COT shot with claude-3-7-sonnet-20250219


100%|███████████████████████████████████████████| 48/48 [04:52<00:00,  6.09s/it]


## Ollama

In [25]:
from langchain_ollama import ChatOllama

In [26]:
MODEL_OLLAMA = [] #["llama3.2:3b-text-fp16"]

In [27]:
def ollama_zero(model):
    model_ = ChatOllama(
        model=model,
        temperature=0,
    )
    chain = build_chian_from_llms(model_)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [28]:
def ollama_make_example(model):
    model = ChatOllama(
        model=model,
        temperature=0,
    )
    chain = build_chian_from_llms(model)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res
    

In [29]:
#display_markdown(ollama_make_example(MODEL_OLLAMA[0]), raw=True)

In [30]:
for model in MODEL_OLLAMA:
    print(f"COT with {model}")
    ollama_zero(model)
    gc.collect()
    torch.mps.empty_cache()

## Huggingface

In [31]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace, HuggingFaceEndpoint
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9DBE5D5C-AC87-30CA-96DA-F5BC116EDA2B> /Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A51C8C05-245A-3989-8D3C-9A6704422CA5> /Users/marcocalamo/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [32]:
MODEL_HUGGINGFACE = [] #["google/gemma-2-2b-it"] #["Qwen/Qwen2.5-3B-Instruct", "microsoft/Phi-3-mini-4k-instruct", "google/gemma-2-27b-it"]

In [33]:
def huggingface_cot(model):
    model_id = model
    
    llm = HuggingFaceEndpoint(
        repo_id=model_id,
        task="text-generation",
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.03,
        temperature=0.01,
)

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = build_chian_from_llms(chat)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model_id.replace("/","_"))

In [34]:
def huggingface_make_example(model):

    model_id = model
    
    

    llm = HuggingFacePipeline.from_model_id(
        model_id=model_id,
        task="text-generation",
        pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 1024}
    )

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = build_chian_from_llms(chat)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
        As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.
        
        In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 
        
        """})
    return res


In [35]:
#display_markdown(huggingface_make_example(MODEL_HUGGINGFACE[0]), raw=True)

In [36]:
for model in MODEL_HUGGINGFACE:
    print(f"COT with {model}")
    huggingface_zero(model)
    gc.collect()
    torch.mps.empty_cache()

## Mlx-LLM

In [37]:
from langchain_community.llms import MLXPipeline
from langchain_community.chat_models import ChatMLX

In [38]:
MODEL_MLX = ["mlx-community/phi-4-8bit",
             "mlx-community/Falcon3-10B-Instruct-8bit", 
             "mlx-community/Qwen2.5-14B-Instruct-4bit",
             "mlx-community/Mistral-7B-Instruct-v0.3-4bit",
             "mlx-community/DeepSeek-R1-Distill-Qwen-7B-8bit",
             "mlx-community/Llama-3.2-3B-Instruct",
             "mlx-community/gemma-2-9b-8bit",
             #"mlx-community/gemma-2-27b-it-4bit",
            # "mlx-community/Mamba-Codestral-7B-v0.1-8bit",
            #"mlx-community/CodeLlama-13b-Instruct-hf-4bit-MLX"
            ]

In [39]:
def mlx_zero(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 30_000, "temp": 0.1},
    )
    chat = ChatMLX(llm=llm)
    chain = build_chian_from_llms(chat)
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model.replace("/","_"))

In [40]:
def mlx_make_example(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 3000, "temp": 0.7},
    )
    chat = ChatMLX(llm=llm)
    chain = build_chian_from_llms(chat)
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res


In [41]:
display_markdown(mlx_make_example(MODEL_MLX[2]), raw=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

@startuml
class InsurancePolicy{}
class Customer{}
class Broker{}
class Contract{}
class Claim{}
class Estimator{}
class Compensation{}

Customer "0..1" -- "1..*" Broker
Customer "0..*" -- "1..*" Contract
Customer "0..*" -- "1..*" Claim
Broker "0..*" -- "0..*" Contract
Estimator "0..*" -- "0..*" Claim
Claim "1..1" -- "0..*" Compensation
@enduml

In [42]:
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [43]:
for model in MODEL_MLX:
    print(f"COT with {model}")
    mlx_zero(model)
    import gc, torch
    gc.collect()
    torch.mps.empty_cache()

COT with mlx-community/phi-4-8bit


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 44/44 [25:02<00:00, 34.16s/it]


COT with mlx-community/Falcon3-10B-Instruct-8bit


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

COT with mlx-community/DeepSeek-R1-Distill-Qwen-7B-8bit


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

  2%|▉                                        | 1/44 [07:00<5:01:17, 420.40s/it]Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x3c30c8ad0>: Failed to resolve \'api.smith.langchain.com\' ([Errno 8] nodename nor servname provided, or not known)"))'))
Content-Length: 5354
API Key: lsv2_********************************************4b
  5%|█▊                                      | 2/44 [28:46<10:59:02, 941.49s/it]Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTT

COT with mlx-community/Llama-3.2-3B-Instruct


100%|███████████████████████████████████████████| 44/44 [23:21<00:00, 31.84s/it]


### Clean Cache

In [44]:
import gc, torch
gc.collect()
torch.mps.empty_cache()

In [45]:
def check_relations(predicted, correct):
    """
        compute precision, recall, f1

        and score from 0 to 5 assigned this way:
            - 1 point if relation exists
            - up to 5 points for the cardinality
    """

    total_score = 0
    true_pos = 0
    tp_fn = len(correct)
    tp_fp = len(predicted)
    
    for rel in correct:
        score = 0
        for rel_ in predicted: 
            if set(rel.keys()) == set(rel_.keys()):
                true_pos += 1
                score += 1
                for r in rel.keys():
                    ## unify cardinality format
                    if ".." not in rel[r]:
                        rel[r] = '"' + rel[r].replace('"','') + ".."+ rel[r].replace('"','') +'"'

                    if ".." not in rel_[r]:
                        rel_[r] = '"' + rel_[r].replace('"','') + ".."+ rel_[r].replace('"','') + '"'
        
                    c_min = rel[r].split("..")[0]
                    c_min_ = rel_[r].split("..")[0]

                    if c_min == c_min_:
                        score+=1

                    c_max = rel[r].split("..")[1]
                    c_max_ = rel_[r].split("..")[1]

                    if c_max == c_max_:
                        score+=1

        total_score += score

    if true_pos > 0 :
        precision = true_pos / tp_fp 
        recall = true_pos / tp_fn 
        f1 = 2 * ((precision * recall) / (precision + recall))
    else:
        precision = 0
        recall = 0
        f1 = 0
    
    return {
        "total_score":total_score,
        "precision": round(precision,2),
        "recall": round(recall,2),
        "f1": round(f1,2),
        "len":len(correct) * 5
    }
            

In [46]:
def check_class(predicted, correct):
    """
    Computes precision, recall, and F1-score between the predicted and correct lists.

    Args:
        predicted (list): The list of predicted values.
        correct (list): The list of correct (ground truth) values.

    Returns:
        dict: A dictionary with precision, recall, and F1-score.
    """

    # precision: how many predicted that are in correct.
    if type(predicted) != type(set()):
        precision =  (len(set(predicted).intersection(set(correct)))) / len(set(predicted))
    else:
        precision =  (len((predicted).intersection((correct)))) / len((predicted))

    
    # recall : how many predicted are not in correct.
    if type(predicted) != type(set()):
        recall =  (len(set(predicted).intersection(set(correct)))) / len(set(correct)) 
    else:
        recall = (len((predicted).intersection((correct)))) / len((correct)) 

    # f1: classic f1
    f1 = 2 * ((precision * recall) / (precision + recall))

    return {
        "precision": round(precision,2),
        "recall": round(recall,2),
        "f1": round(f1,2)
    }

In [47]:
def init_parser():
    with open("/Users/marcocalamo/Downloads/grammar.ebnf", encoding="utf-8") as grammar_file:
        parser = Lark(grammar_file.read())
        return parser
        
def parse_text(parser, ref):
    ref = ref.replace('`','')
    ref = ref.replace('plantuml','')
    ref = ref.replace('plaintext', '')

    if not '@startuml' in ref:
        ref = '@startuml\n' + ref
    if not '@enduml' in ref:
        ref = ref+'\n@enduml'
    #ref = ref.replace('@startuml','')
    #ref = ref.replace('@enduml','')
    parsed_ref = parser.parse(ref)
    return parsed_ref

def get_from_parsed(parsed_text, to_get="class"):
    found = parsed_text.find_pred(lambda v: v.data.value == to_get)
    
    to_ret = []

    for f in found:
        if to_get == 'relationship':
            one_rel = f.children[0].children            
            first_rel = one_rel[0].children[0].value
            assert one_rel[0].children[0].type == 'CNAME'

            try:
                first_card = one_rel[1].children[0].value
                assert one_rel[1].children[0].type == 'ESCAPED_STRING'
            except:
                first_card = "0..*"

            try:
                second_card = one_rel[5].children[0].value
                assert one_rel[5].children[0].type == 'ESCAPED_STRING'
            except:
                second_card = "0..*"

            second_rel = one_rel[-2].children[0].value
            assert one_rel[-2].children[0].type == 'CNAME'
            
            rel_tuple = {
                first_rel:first_card,
                second_rel:second_card
            }
            
            to_ret.append(rel_tuple)
        elif to_get == 'class':
            to_ret.append(f.children[0].children[0].value)
    
    return to_ret
    
def parse_path(path_to_uml, parser):
    
    if os.path.isfile(path_to_uml):
        with open(path_to_uml) as plant_ref:
            
            ref = plant_ref.read()
            parsed_ref = parse_text(parser, ref)
            found_class = get_from_parsed(parsed_ref, to_get="class")
            found_rel = get_from_parsed(parsed_ref, to_get="relationship")

            return found_class, found_rel
    else:
        raise FileNotFoundError
    

In [48]:
from lark import Lark
import os

In [49]:
#parse_path("/Users/marcocalamo/KUL/Dataset/AuthorsAndBooks/uml.txt", init_parser())

In [50]:
#parse_path("/Users/marcocalamo/KUL/Dataset/Alpha_Insurance_Basic/result_cot_gpt-3.5-turbo.txt", init_parser())

In [51]:
def evaluate(path_uml, path_to_check, parser):

    # gold class and rel
    class_uml, rel_uml = parse_path(path_uml, parser)

    # candidate class and rel
    class_check, rel_check = parse_path(path_to_check, parser)

    # check class
    res_cl = check_class(class_check, class_uml)

    # check rel
    res_re = check_relations(rel_check, rel_uml)

    return res_cl, res_re

In [52]:
#evaluate("/Users/marcocalamo/KUL/Dataset/AuthorsAndBooks/uml.txt", "/Users/marcocalamo/KUL/Dataset/AuthorsAndBooks/result_cot_mlx-community_phi-4-8bit.txt",init_parser())

In [53]:
import os
import csv

def process_folders(root_folder):
    csv_file = os.path.join(root_folder, "evaluation_results.csv")
    header_written = False
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        for sub_folder in os.listdir(root_folder):
            sub_folder_path = os.path.join(root_folder, sub_folder)
            if not os.path.isdir(sub_folder_path):
                continue

            uml_file = os.path.join(sub_folder_path, "uml.txt")
            if not os.path.exists(uml_file):
                continue
            
            for filename in os.listdir(sub_folder_path):
                if filename.startswith("result_") and filename.endswith(".txt"):
                    result_file = os.path.join(sub_folder_path, filename)
                    model_name = "-".join(filename.replace(".txt", "").split("_")[1:])
                    parser = init_parser()
                    try:
                        results = evaluate(uml_file, result_file, parser)
                        if not header_written:
                            writer.writerow(["sub_folder_name", "model_name", 
                                             "precision_class", "recall_class", "f1_class",
                                             "precision_rel", "recall_rel", "f1_rel", "score_rel", "max_score"])
                            header_written = True
                        writer.writerow([sub_folder, model_name, results[0]["precision"], results[0]["recall"], results[0]["f1"],
                                        results[1]["precision"], results[1]["recall"], results[1]["f1"],
                                        results[1]["total_score"], results[1]["len"]])
                    except Exception as e:
                        print(e,sub_folder, model_name)
                        with open("trace.txt", mode="a") as t:
                            t.write(str(e) + str(sub_folder)+str (model_name))
                        #raise Exception

In [54]:
process_folders(ROOT_FOLDER)

No terminal matches '1' in the current parser context, at line 4 col 9

Project 1..* -- 1..1 Employee
        ^
Expected one of: 
	* COLON
	* __ANON_16
	* __ANON_20
	* __ANON_18
	* __ANON_22
	* __ANON_19
	* SPACE
	* __ANON_23
	* __ANON_15
	* __ANON_21
	* ESCAPED_STRING
	* MINUS
	* STAR
	* __ANON_1
	* ENTITY
	* __ANON_5
	* __ANON_17
	* O
	* __ANON_14
	* DOT
	* __ANON_12
	* __ANON_4
	* HASH
	* SKINPARAM
	* LESSTHAN
	* CNAME
	* RBRACE
	* CIRCUMFLEX
	* STATE
	* X
	* PLUS
	* CLASS
 EmployeesAndLeaders cot-mlx-community-Falcon3-10B-Instruct-8bit
No terminal matches '<' in the current parser context, at line 1 col 1

<think>
^
Expected one of: 
	* __ANON_0
 EmployeesAndLeaders mlx-community-DeepSeek-R1-Distill-Qwen-7B-8bit
No terminal matches '"' in the current parser context, at line 6 col 16

Project "1" -- "* Employee
               ^
Expected one of: 
	* ESCAPED_STRING
	* RBRACE
	* CIRCUMFLEX
	* MORETHAN
	* STAR
	* X
	* SPACE
	* __ANON_13
	* O
	* HASH
	* PLUS
	* CNAME
 EmployeesAndLeaders

In [55]:
def add_counts(ax, df):
    for c in ax.containers:
        # Extract heights and match with correct counts
        labels = [f'N={int(row["count"])}' for (idx, row) in (df.iterrows()) ]
        # Add the count labels to the bars
        try:
            ax.bar_label(c, labels=labels, label_type='edge', fontsize=10, padding=3)
        except:
            continue

In [56]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_data_class(csv):
    # Load the CSV file
    df = pd.read_csv(csv)
    
    # Extract the base model name and determine result type
    df['base_model_name'] = df['model_name'].apply(lambda x: x.replace('cot-', ''))
    df['result_type'] = df['model_name'].apply(lambda x: 'cot' if 'cot-' in x else 'standard')
    
    # Compute averages and instance counts
    avg_metrics = df.groupby(['base_model_name', 'model_name', 'result_type']).agg(
        f1_class_avg=('f1_class', 'mean'),
        count=('model_name', 'count')
    ).reset_index()

    
    # Set seaborn style
    sns.set_theme(style='whitegrid')
    
    # Define a color palette for base model names
    unique_base_models = avg_metrics['base_model_name'].unique()
    base_palette = sns.color_palette("tab10", len(unique_base_models))
    color_map = {model: base_palette[i] for i, model in enumerate(unique_base_models)}
    
    # Generate colors for the bars, ensuring each base model has the same color for both result types
    avg_metrics['color'] = avg_metrics['base_model_name'].map(color_map)
    
    # Plot average f1_class per model with result types
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(
        x='base_model_name', y='f1_class_avg', hue='result_type', 
        data=avg_metrics, palette='husl' #palette=[color_map[m] for m in avg_metrics['base_model_name']]
    )

    # Annotate counts on bars correctly
    for count, score in zip(avg_metrics['count'], avg_metrics['f1_class_avg']):        
        for p in ax.patches:
            if p.get_height() == score:
                ax.annotate(f'N={count}', 
                            (p.get_x() + p.get_width() / 2., p.get_height()), 
                            ha='center', va='bottom', fontsize=10)
    
    
    plt.title('Average F1 Class per Model')
    plt.xlabel('Model Name')
    plt.ylabel('Average F1 Class')
    plt.xticks(rotation=30, ha='right')
    plt.legend(title='Result Type')
    plt.show()


def plot_data_rel(csv):
    df = pd.read_csv(csv)
    
    # Extract the base model name and determine result type
    df['base_model_name'] = df['model_name'].apply(lambda x: x.replace('cot-', ''))
    df['result_type'] = df['model_name'].apply(lambda x: 'cot' if 'cot-' in x else 'standard')
    
    # Compute averages and instance counts
    avg_metrics = df.groupby(['base_model_name', 'model_name', 'result_type']).agg(
        f1_rel_avg=('f1_rel', 'mean'),
        count=('model_name', 'count')
    ).reset_index()
    
    # Set seaborn style
    sns.set_theme(style='whitegrid')
    
    # Define a color palette for base model names
    unique_base_models = avg_metrics['base_model_name'].unique()
    base_palette = sns.color_palette("tab10", len(unique_base_models))
    color_map = {model: base_palette[i] for i, model in enumerate(unique_base_models)}
    
    # Generate colors for the bars, ensuring each base model has the same color for both result types
    avg_metrics['color'] = avg_metrics['base_model_name'].map(color_map)
    
    # Plot average f1_class per model with result types
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(
        x='base_model_name', y='f1_rel_avg', hue='result_type', 
        data=avg_metrics, palette='husl' #palette=[color_map[m] for m in avg_metrics['base_model_name']]
    )

    # Annotate counts on bars correctly
    for count, score in zip(avg_metrics['count'], avg_metrics['f1_rel_avg']):        
        for p in ax.patches:
            if p.get_height() == score:
                ax.annotate(f'N={count}', 
                            (p.get_x() + p.get_width() / 2., p.get_height()), 
                            ha='center', va='bottom', fontsize=10)
        
    plt.title('Average F1 Association per Model')
    plt.xlabel('Model Name')
    plt.ylabel('Average F1 Assoc')
    plt.xticks(rotation=30, ha='right')
    plt.legend(title='Result Type')
    plt.show()


def plot_data_score(csv):
    # Load the CSV file
    df = pd.read_csv(csv)
    
    # Extract the base model name and determine result type
    df['base_model_name'] = df['model_name'].apply(lambda x: x.replace('cot-', ''))
    df['result_type'] = df['model_name'].apply(lambda x: 'cot' if 'cot-' in x else 'standard')
    
    # Compute averages and instance counts
    avg_metrics = df.groupby(['base_model_name', 'model_name', 'result_type']).agg(
        score_rel=('score_rel', 'mean'),
        count=('model_name', 'count')
    ).reset_index()
    
    # Set seaborn style
    sns.set_theme(style='whitegrid')
    
    # Define a color palette for base model names
    unique_base_models = avg_metrics['base_model_name'].unique()
    base_palette = sns.color_palette("tab10", len(unique_base_models))
    color_map = {model: base_palette[i] for i, model in enumerate(unique_base_models)}
    
    # Generate colors for the bars, ensuring each base model has the same color for both result types
    avg_metrics['color'] = avg_metrics['base_model_name'].map(color_map)
    
    # Plot average f1_class per model with result types
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(
        x='base_model_name', y='score_rel', hue='result_type', 
        data=avg_metrics, palette='husl' #palette=[color_map[m] for m in avg_metrics['base_model_name']]
    )
    # Annotate counts on bars correctly
    for count, score in zip(avg_metrics['count'], avg_metrics['score_rel']):        
        for p in ax.patches:
            if p.get_height() == score:
                ax.annotate(f'N={count}', 
                            (p.get_x() + p.get_width() / 2., p.get_height()), 
                            ha='center', va='bottom', fontsize=10)
    
    plt.title('Average Score per Model')
    plt.xlabel('Model Name')
    plt.ylabel('Average Score')
    plt.xticks(rotation=30, ha='right')
    plt.legend(title='Result Type')
    plt.show()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [57]:
#plot_data_class("/Users/marcocalamo/KUL/Dataset/evaluation_results.csv")

In [58]:
#plot_data_rel("/Users/marcocalamo/KUL/Dataset/evaluation_results.csv") 

In [59]:
#plot_data_score("/Users/marcocalamo/KUL/Dataset/evaluation_results.csv")